In [16]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

# 1. First, check class distribution in your dataset
dataset_path = r"C:\College\dsmlcourse\DSML projects\dataset\model security dataset"

# Print number of images in each class folder
for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):
        num_images = len([f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))])
        print(f"Class '{class_name}': {num_images} images")

# 2. Rebuild your model with class weights to handle imbalance
# Calculate class weights (more weight to underrepresented classes)
total_samples = sum([len(os.listdir(os.path.join(dataset_path, d))) 
                     for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))])
class_weights = {}
for i, class_name in enumerate(os.listdir(dataset_path)):
    if os.path.isdir(os.path.join(dataset_path, class_name)):
        n_samples = len(os.listdir(os.path.join(dataset_path, class_name)))
        class_weights[i] = (1 / n_samples) * (total_samples / 3)  # 3 classes

print("Class weights:", class_weights)

# 3. Load the dataset with the EXACT same preprocessing for both training and inference
train_ds = keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(224, 224),
    batch_size=15,
    validation_split=0.2,
    subset="training",
    seed=42
)

val_ds = keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(224, 224),
    batch_size=15,
    validation_split=0.2,
    subset="validation",
    seed=42
)

# Save the EXACT class names from the dataset
class_names = train_ds.class_names
print("Actual class names from dataset:", class_names)

# 4. Create a preprocessing layer that will be part of the model
preprocessing = keras.Sequential([
    keras.layers.Rescaling(1./127.5, offset=-1)  # MobileNetV2 preprocessing
])

# Apply preprocessing consistently
train_ds = train_ds.map(lambda x, y: (preprocessing(x), y))
val_ds = val_ds.map(lambda x, y: (preprocessing(x), y))

# 5. Rebuild model with more capacity and regularization
base_model = keras.applications.MobileNetV2(
    weights="imagenet", 
    include_top=False, 
    input_shape=(224, 224, 3)
)
base_model.trainable = False  # Freeze base model

model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(256, activation="relu"),  # Larger intermediate layer
    keras.layers.Dropout(0.5),  # Strong dropout
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(len(class_names), activation="softmax")
])

# 6. Compile with better optimization parameters
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),  # Lower learning rate
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# 7. Train with class weights to handle imbalance
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    class_weight=class_weights,  # Apply class weights
    callbacks=[
        keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=3)
    ]
)

# 8. Save model and class names together
save_dir = r"C:\College\dsmlcourse\model_security_2"
os.makedirs(save_dir, exist_ok=True)

# Save class names to text file
with open(os.path.join(save_dir, "class_names.txt"), "w") as f:
    for name in class_names:
        f.write(name + "\n")

# Convert to TFLite with optimization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open(os.path.join(save_dir, "model.tflite"), "wb") as f:
    f.write(tflite_model)

print("Model and class names saved to:", save_dir)

Class 'firesmoke': 100 images
Class 'medical assist': 192 images
Class 'normal': 192 images
Class 'trackdefective': 147 images
Class 'tracknotdefective': 201 images
Class weights: {0: 2.773333333333333, 1: 1.4444444444444442, 2: 1.4444444444444442, 3: 1.8866213151927436, 4: 1.3797678275290215}
Found 832 files belonging to 5 classes.
Using 666 files for training.
Found 832 files belonging to 5 classes.
Using 166 files for validation.
Actual class names from dataset: ['firesmoke', 'medical assist', 'normal', 'trackdefective', 'tracknotdefective']
Epoch 1/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 19s 296ms/step - accuracy: 0.4514 - loss: 2.5489 - val_accuracy: 0.7771 - val_loss: 0.4875 - learning_rate: 5.0000e-04
Epoch 2/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 12s 269ms/step - accuracy: 0.8223 - loss: 0.7717 - val_accuracy: 0.9096 - val_loss: 0.2612 - learning_rate: 5.0000e-04
Epoch 3/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 12s 271ms/step - accuracy: 0.8551 - loss: 0.6516 - val_accuracy: 0.9337 - val_loss: 0.2146 - lear

INFO:tensorflow:Assets written to: C:\Users\nadar\AppData\Local\Temp\tmpjcy35ajy\assets


Saved artifact at 'C:\Users\nadar\AppData\Local\Temp\tmpjcy35ajy'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_816')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  2630053108176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2631282658384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2631282657232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2631282658192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2631282658576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2631282658960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2631282660304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2631282660496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2631282660112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2631282659344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2

In [ ]:
import tensorflow.lite as tflite
import numpy as np
from tensorflow.keras.preprocessing import image
import os

# Load the proper class names from your saved file
class_names_path = r"C:\College\dsmlcourse\model_security_2\class_names.txt"
with open(class_names_path, "r") as f:
    class_names = [line.strip() for line in f.readlines()]

print("Using these class names:", class_names)

# Load the TFLite model
interpreter = tflite.Interpreter(model_path=r"C:\College\dsmlcourse\model_security_2\model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def predict_image(img_path):
    # Load and preprocess image exactly as in training
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Apply the EXACT SAME preprocessing as during training
    img_array = img_array / 127.5 - 1  # MobileNetV2 preprocessing
    
    # Run inference
    interpreter.set_tensor(input_details[0]['index'], img_array)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    
    # Get predicted class and confidence
    predicted_class = np.argmax(output[0])
    confidence = output[0][predicted_class]
    
    # Print all class probabilities for debugging
    for i, prob in enumerate(output[0]):
        print(f"  {class_names[i]}: {prob:.4f}")
    
    return predicted_class, confidence

# Test multiple images
test_images = [
r"C:\Users\nadar\Downloads\download.jpg"
]

for img_path in test_images:
    print(f"\nPredicting: {os.path.basename(img_path)}")
    predicted_class, confidence = predict_image(img_path)
    print(f"Predicted class: {class_names[predicted_class]}")
    print(f"Confidence: {confidence:.4f}")

Using these class names: ['firesmoke', 'medical assist', 'normal', 'trackdefective', 'tracknotdefective']

Predicting: download.jpg
  firesmoke: 0.9979
  medical assist: 0.0003
  normal: 0.0002
  trackdefective: 0.0014
  tracknotdefective: 0.0001
Predicted class: firesmoke
Confidence: 0.9979
